In [1]:
# Import Libraries and Dependencies
import pandas as pd

### Read in files

In [2]:
# Read in data and index by CustomerID
jan = pd.read_csv('Resources/Jan2019_sales.csv', index_col='order_ID')
feb = pd.read_csv('Resources/Feb2019_sales.csv', index_col='order_ID')
mar = pd.read_csv('Resources/March2019_sales.csv', index_col='order_ID')

In [3]:
# Show sample of January sales data.
jan.head(5)

,product,quantity_ordered,price,order_date,state,city,zip_code
order_ID,,,,,,,
141234,iPhone,1,700.00,01/22/19 21:25,FL,Orlando,32807
141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,FL,Miami,33137
141236,Wired Headphones,2,11.99,01/17/19 13:33,GA,Atlanta,30318
141237,27in FHD Monitor,1,149.99,01/05/19 20:33,CA,Los Angeles,90014
141238,Wired Headphones,1,11.99,01/25/19 11:59,CA,San Francisco,94016


In [4]:
# Show sample of February sales data.
feb.head(5)

,product,quantity_ordered,price,order_date,state,city,zip_code
order_ID,,,,,,,
150502,iPhone,1,700.00,02/18/19 01:35,PA,Philadelphia,19132
150503,AA Batteries (4-pack),1,3.84,02/13/19 07:24,CA,San Diego,91911
150504,27in 4K Gaming Monitor,1,389.99,02/18/19 09:46,CA,San Diego,91911
150505,Lightning Charging Cable,1,14.95,02/02/19 16:47,FL,Orlando,32822
150506,AA Batteries (4-pack),2,3.84,02/28/19 20:32,WA,Seattle,98115


In [5]:
# Show sample of March sales data.
mar.head(5)

,product,quantity_ordered,price,order_date,state,city,zip_code
order_ID,,,,,,,
162009,iPhone,1,700.00,03/28/19 20:59,CA,Los Angeles,90008
162009,Lightning Charging Cable,1,14.95,03/28/19 20:59,TX,Austin,78729
162009,Wired Headphones,2,11.99,03/28/19 20:59,OR,Portland,97209
162010,Bose SoundSport Headphones,1,99.99,03/17/19 05:39,FL,Miami,33125
162011,34in Ultrawide Monitor,1,379.99,03/10/19 00:01,CA,San Diego,92029


### Check the data types of each DataFrame

In [6]:
# Jan sales datatypes
jan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9681 entries, 141234 to 150501
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product           9681 non-null   object 
 1   quantity_ordered  9681 non-null   int64  
 2   price             9681 non-null   float64
 3   order_date        9681 non-null   object 
 4   state             9681 non-null   object 
 5   city              9681 non-null   object 
 6   zip_code          9681 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 605.1+ KB


In [7]:
# Feb sales datatypes
feb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11986 entries, 150502 to 162008
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product           11986 non-null  object 
 1   quantity_ordered  11986 non-null  int64  
 2   price             11986 non-null  float64
 3   order_date        11986 non-null  object 
 4   state             11986 non-null  object 
 5   city              11986 non-null  object 
 6   zip_code          11986 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 749.1+ KB


In [8]:
# March sales datatypes
mar.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15154 entries, 162009 to 176557
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product           15154 non-null  object 
 1   quantity_ordered  15154 non-null  int64  
 2   price             15154 non-null  float64
 3   order_date        15154 non-null  object 
 4   state             15154 non-null  object 
 5   city              15154 non-null  object 
 6   zip_code          15154 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 947.1+ KB


### Combine the sales data by rows.

In [9]:
# Combine the January, February, and March sales DataFrames on the rows using an inner join, and reset the index
sales = pd.concat([jan, feb,mar], axis='rows',join = 'inner', ignore_index=False).reset_index()

#unit test - check if joined properly
total_count = jan.shape[0] + feb.shape[0] + mar.shape[0]
total_joined_count = sales.shape[0]
if total_count == total_joined_count:
    print(f"joined properly  {total_count} == {total_joined_count}")
else:
    print(f'joined inccorectely {total_joined_count} != {total_count}')

sales.head(2)

joined properly  36821 == 36821


,order_ID,product,quantity_ordered,price,order_date,state,city,zip_code
0,141234,iPhone,1,700.00,01/22/19 21:25,FL,Orlando,32807
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,FL,Miami,33137


In [10]:
# Check if any values are null.
sales.isna().sum()

order_ID            0
product             0
quantity_ordered    0
price               0
order_date          0
state               0
city                0
zip_code            0
dtype: int64

In [11]:
# Check the data type of each column
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36821 entries, 0 to 36820
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_ID          36821 non-null  int64  
 1   product           36821 non-null  object 
 2   quantity_ordered  36821 non-null  int64  
 3   price             36821 non-null  float64
 4   order_date        36821 non-null  object 
 5   state             36821 non-null  object 
 6   city              36821 non-null  object 
 7   zip_code          36821 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 2.2+ MB


In [12]:
# Convert the "order_date" column to a datetime datatype.
sales['order_date'] = pd.to_datetime(sales['order_date'], format='%m/%d/%y %H:%M')




In [13]:
# Confirm that the data type has been changed.
sales['order_date'].head()

0   2019-01-22 21:25:00
1   2019-01-28 14:15:00
2   2019-01-17 13:33:00
3   2019-01-05 20:33:00
4   2019-01-25 11:59:00
Name: order_date, dtype: datetime64[ns]

### 1. Which top five zip codes in which cities and states had the greatest number of products ordered?

### Using `groupby`

In [14]:
# Show the average number items ordered for each state, city, and zip code.
def agg_values(df, cat_col,num_col,stat):
    
    """
    Returns aggrigated data frame
    args:
        df - Data Frame
        cat_col - catagorical columns to groupby
        num_col - numerical columns to aggrigate
        stat - type of aggrigation
        n - top values
    """
    aggs  = df.groupby(cat_col).agg({num_col:stat}).reset_index()
    aggs_sorted = aggs.sort_values(num_col,ascending= False)
    
    return aggs_sorted.reset_index(drop = True)

avg_items_ordered = agg_values(sales, ['zip_code','city','state'],'quantity_ordered','mean')
# Rename the sum column to reflect the data in the column.
avg_items_ordered.rename(columns={'quantity_ordered':'avg_items_ordered'},inplace= True)


# Show the top 5 results.
round(avg_items_ordered,2).head()



,zip_code,city,state,avg_items_ordered
0,60629,Chicago,IL,1.17
1,94102,San Francisco,CA,1.15
2,33125,Miami,FL,1.15
3,48215,Detroit,MI,1.15
4,94016,San Francisco,CA,1.15


### Using `pivot_table`

In [15]:
# Show the total number of items ordered for each state, city, and zip code.
def agg_pivot(df,cat_col,num_col,stat):

       pivot_table = pd.pivot_table(df, values=num_col,\
                              index=cat_col, aggfunc= stat)
       
       return pivot_table.sort_values(num_col,ascending= False).reset_index()
#---------------------------------------------------------------------------------------------------

items_ordered_pivot = agg_pivot(sales,['zip_code','city','state'],'quantity_ordered','sum')


# Rename the "sum" column to "Avg_Number_Products_Ordered"

items_ordered_pivot.rename(columns={'quantity_ordered':'total_items_ordered'},inplace= True)

# Show the top 5 results.
items_ordered_pivot.head()


,zip_code,city,state,total_items_ordered
0,60629,Chicago,IL,909
1,92029,San Diego,CA,873
2,91911,San Diego,CA,870
3,10001,New York City,NY,866
4,43201,Columbus,OH,866


### 2. Which top five zip codes in which cities and states generated the most sales?

### Using `groupby`

In [16]:
# Show the total price for the items ordered for each state, city, and zip code.
sales['total_price'] = sales['price']* sales['quantity_ordered']

total_sales_price = agg_values(sales,['zip_code','city','state'],'total_price','sum')

#Format price
total_sales_price['total_price'] = total_sales_price['total_price'].map('${:,.2f}'.format)

# Rename the sum column to reflect the data in the column.
total_sales_price.rename(columns={'total_price':'total_sales_price'},inplace= True)

# Show the top 5 results.

total_sales_price.head()


,zip_code,city,state,total_sales_price
0,14201,Buffalo,NY,"$156,582.83"
1,60629,Chicago,IL,"$151,721.31"
2,92029,San Diego,CA,"$151,342.97"
3,94102,San Francisco,CA,"$146,945.58"
4,10013,New York City,NY,"$146,367.28"


### Using `pivot_table`

In [17]:
# Show the total price for all of the items ordered for each state, city, and zip code.
total_sales_pivot = agg_pivot(sales,['zip_code','city','state'],'total_price','sum')

# Rename the "sum" column to "Avg_Number_Products_Ordered"
total_sales_pivot['total_price'] = total_sales_pivot['total_price'].map('${:,.2f}'.format)
total_sales_pivot.rename(columns={'quantity_ordered':'total_sales_price'},inplace= True)

# Show the top 5 results.
total_sales_pivot.head()


,zip_code,city,state,total_price
0,14201,Buffalo,NY,"$156,582.83"
1,60629,Chicago,IL,"$151,721.31"
2,92029,San Diego,CA,"$151,342.97"
3,94102,San Francisco,CA,"$146,945.58"
4,10013,New York City,NY,"$146,367.28"


### 3. Which top five zip codes in which cities and states had the greatest average price for the products ordered?

### Using `groupby`

In [18]:
# Show the average price of items ordered for each state, city, and zip code rounded to two decimal places.
avg_price = agg_values(sales,['zip_code','city','state'],'price','mean')

# Rename the mean column to reflect the data in the column.
avg_price.rename(columns={'price':'avg_price'},inplace= True)

#Round and format price
avg_price['avg_price'] = round(avg_price['avg_price'],2)
avg_price['avg_price'] = avg_price['avg_price'].map('${:,.2f}'.format)

# Show the top 5 results.
avg_price.head()

,zip_code,city,state,avg_price
0,10013,New York City,NY,$212.11
1,94102,San Francisco,CA,$210.85
2,78745,Austin,TX,$205.63
3,14201,Buffalo,NY,$203.89
4,92029,San Diego,CA,$196.73


### Using `pivot_table`

In [19]:
# Show the average price per product ordered for each state, city, and zip code rounded to two decimal places.

avg_price_pivot = agg_pivot(sales,['zip_code','city','state'],'price','mean')

# Rename the "sum" column to "Avg_Number_Products_Ordered"
avg_price_pivot['price'] = avg_price_pivot['price'].map('${:,.2f}'.format)
avg_price_pivot.rename(columns={'price':'avg_sales_price'},inplace= True)

# Show the top 5 results.
avg_price_pivot.head()



,zip_code,city,state,avg_sales_price
0,10013,New York City,NY,$212.11
1,94102,San Francisco,CA,$210.85
2,78745,Austin,TX,$205.63
3,14201,Buffalo,NY,$203.89
4,92029,San Diego,CA,$196.73


### 4. Which top five zip codes in which cities and states had the most iPhone sales?

In [20]:
# Filter the sales data to get the iPhone sales data.


iphone_sales = sales[sales['product'].str.contains('iphone',case=False)]

totap_iphone_qty = agg_values(iphone_sales,['zip_code'],'quantity_ordered','sum')

totap_iphone_qty.head()

,zip_code,quantity_ordered
0,10001,40
1,10013,36
2,14201,35
3,78701,34
4,94102,33


### Using `groupby`

In [21]:
# Show the total number of items ordered for each state, city, and zip code.
total_iphone_ordered = agg_values(iphone_sales,['zip_code','city','state'],'quantity_ordered','sum')
# Rename the count column to reflect the data in the column.
total_iphone_ordered.rename(columns={'quantity_ordered':'total_items_ordered'},inplace=True)

#Show the top 5 results.
total_iphone_ordered.head()

,zip_code,city,state,total_items_ordered
0,10001,New York City,NY,40
1,10013,New York City,NY,36
2,14201,Buffalo,NY,35
3,78701,Austin,TX,34
4,94102,San Francisco,CA,33


### Using `pivot_table`

In [22]:
# Show the total number of items ordered for each city and zip code.
total_iphone_ordered_pivot = agg_pivot(iphone_sales,['zip_code','city','state'],'quantity_ordered','sum')
# Rename the "quantity_ordered" column to "Number_Ordered"

total_iphone_ordered_pivot.rename(columns={'quantity_ordered':'total_items_ordered'},inplace=True)
# Show the top 5 results.
total_iphone_ordered_pivot.head()

,zip_code,city,state,total_items_ordered
0,10001,New York City,NY,40
1,10013,New York City,NY,36
2,14201,Buffalo,NY,35
3,78701,Austin,TX,34
4,94102,San Francisco,CA,33


### 5. Which day had the most iPhone sales?

In [23]:
# Create a pivot table with the 'order_date' as the index and the "price" as the values.

date_price_pivot  = agg_pivot(sales, 'order_date','price','sum')

# Rename the "price" column to "Total_Sales"
date_price_pivot.rename(columns={'price':'Total_sales'},inplace=True)



# Show the table.

date_price_pivot.set_index('order_date').head()

,Total_sales
order_date,
2019-03-29 21:17:00,3414.95
2019-01-20 00:15:00,3400.00
2019-02-22 21:06:00,3400.00
2019-02-26 17:12:00,3400.00
2019-03-30 10:47:00,3400.00


In [24]:
# Resample the pivot table into daily bins 
date_price_pivot['date'] = date_price_pivot['order_date'].dt.date


# and get the total sales for each day rounded to two decimal places.
daily_items_order = agg_values(date_price_pivot,'date', 'Total_sales','sum')

# Sort the resampled pivot table in ascending order on "Total_Sales".
daily_items_order.sort_values('Total_sales',ascending= False).reset_index().head()

,index,date,Total_sales
0,0,2019-03-22,102516.59
1,1,2019-03-25,101101.30
2,2,2019-03-18,100291.58
3,3,2019-03-05,99455.47
4,4,2019-03-26,98513.15


### 6.  Which week had the most iPhone sales?

In [33]:
# Resample the pivot table into weekly bins 
# and get the total sales for each week rounded to two decimal places.
iphone_sales['week'] = iphone_sales['order_date'].dt.isocalendar().week

weekly_sales = agg_values(iphone_sales,'week','total_price','sum')
# Sort the resampled pivot table in ascending order on "Total_Sales"
weekly_sales.sort_values('total_price').head().reset_index(drop = True)


C:\Users\hafto\AppData\Local\Temp\ipykernel_20072\388982888.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iphone_sales['week'] = iphone_sales['order_date'].dt.isocalendar().week


,week,total_price
0,3,46200.0
1,1,47600.0
2,2,59500.0
3,8,67900.0
4,7,74200.0
